In [ ]:
import csv
from functools import reduce
import json

# Fungsi untuk memvalidasi apakah string dapat dikonversi ke float
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Baca data dari file CSV dan filter
data_list = []
with open('/content/911_Calls_for_Service_(Last_30_Days).csv', mode='r') as file:
    reader = csv.DictReader(file)
    data_list = [row for row in reader]

# Filter dictionary yang memiliki kolom Zip dan Neighborhood serta kolom waktu yang valid
filtered_data = list(filter(lambda x: x['zip_code'] and x['neighborhood'] and is_float(x['totalresponsetime']) and is_float(x['dispatchtime']) and is_float(x['totaltime']), data_list))
